In [43]:
with open('../../my_key.txt') as f:
    key_list = f.readlines()

key_list = [key.strip() for key in key_list]

keys = {}
for key in key_list:
    key = key.split(':')
    if (key != ['']):
        keys[key[0]] = key[1]

In [44]:
import os

os.environ['OPENAI_API_KEY'] = keys['OPENAI_API_KEY']

# One Call Information Engine
- Complete the information summarization with one call
- I/O
    - Input: text to summarization
    - Output: Structured Information

In [ ]:
with open('text_to_su.txt', 'r') as f:
    text_to_su = f.read()
# print(text_to_su)

## Tiktoken

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [ ]:
num_tokens = len(encoding.encode(text_to_su))
print(num_tokens)

## One Call Information Engine

In [45]:
import openai

openai.organization = keys['OPENAI_ORGANIZATION']
openai.api_key = os.getenv('OPENAI_API_KEY')

In [84]:
sys_prompt = """You are a summary writer."""

instruction1 = """Write a summary on those web text. You may make some comparisons on the text from different sources."""
instruction2 = "Yes, I will write it with sequence number 1,2,3"

usr_prompt = f"""
<Web texts: (The content is from web so there might be some noise, you should ignore them)
{text_to_su}>

<Each point you conclude must include the source and output in Sequence, for example:
1 "Point a"[url1]\n
1.1 "Point b"[url1][url2]\n
2 "point c"[url3]>\n

Summary: (Write in detail and with reference, remember to include the source)
"""

response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role": "system", "content": f'{sys_prompt}'},
        {"role": 'user', "content": instruction1},
        {"role": 'assistant', "content": instruction2},
        {"role": 'user', 'content': f'{usr_prompt}'},
    ],
    # max_tokens = 2000,
    # stream = True
)

#for chunk in tur_response:
#    try:
#        print(chunk['choices'][0]['delta']['content'])
#    except:
#        pass

In [85]:
response

<OpenAIObject chat.completion id=chatcmpl-7MApIcPRVqhgjSFEzP8z7FmCln3zF at 0x7fe28e765190> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "1. Both GPT and BERT are natural language processing (NLP) models that can perform a range of NLP tasks, like language generation, text classification, and question answering.\n1.1 \"GPT-3 (Generative Pre-trained Transformer 3) is an autoregressive language model developed by OpenAI. It is a language model that can generate natural language text, answer questions, compose poetry, and write complete articles. It has 175 billion parameters, making it one of the largest language models ever constructed.\" [url0]\n1.2 \"BERT (Bidirectional Encoder Representations from Transformers) is a pre-training language representation model that fine-tunes NLP applications created by Google in 2018. BERT uses bidirectional flow, which allows it to use context from both directions during processing

In [86]:
with open('result.txt', 'w') as f:
    f.write(response['choices'][0]['message']['content'])
